# Satellite Environment with OpenAI Gym

In [1]:
from gym import Env
from gym.spaces import *
import numpy as np
import random
import matplotlib.pyplot as plt

In [109]:
"""
State of satellite
P: power of satellite
S1: memory in sensor 1
S2: memory in sensor 2
S3: memory in sensor 3
E: electronics of satellite
O: orbital motion of satellite
T: time of the day-> ionosphere behavior

State Space:
S = (P, S1, S2, S3, E, O, T)
P = {100, 99, 98, …, 3, 2, 1, 0}
S1 = {100, 99, 98, …, 3, 2, 1, 0}
S2 = {100, 99, 98, …, 3, 2, 1, 0}
S3 = {100, 99, 98, …, 3, 2, 1, 0}
E = {0, 1} not working / working
O = {0, 1} not in range / in range
T = {0, 1, 2, 3, 4} night, dawn, morning, afternoon, dusk

Action Space:
A = {transmit, not transmit} = {1, 0}

Step Function:
*If a1= transmit = 1, then:
    P(state=s) transitions to a lower value P(state=s’) ---> power discharges
    Ex: P=100 -> a1 (transmit) -> P=95
    S1(state=s) transitions to a lower value S1(state=s’) ---> free memory in sensor 1 decreases (data transferred)
    Ex: S1=100 -> a1 (transmit) -> S1=96.5

*If a1= not transmit = 0, then:
    P(state=s) transitions to a higher value P(state=s’) ---> power charges
    Ex: P=70 -> a1 (not transmit) -> P=73
    S1(state=s) remains the same S1(state=s’) ---> free memory in sensor 1 constant (no data transferred)
    Ex: S1=70 -> a1 (not transmit) -> S1=70

Reward:
*If a1=1 (transmit), then:    ---> For each transmission we get reward
    reward +=5 
*If P(state=s’)>=30, then:    ---> Rewards for maintaining power more than 30%
    reward +=2 
*If we transmit all the data within the time satellite is within range then:
    reward += 5*(remaining time for satellite in range)  --->More rewards for quicker transmission

Termination:
An action can occur every minute for an hour (when the satellite is within range) or until the sensor memory is full i.e. terminate if:
*S1(state=s) = 0 --->Transmission complete
*O(state=s) = 0  --->Satellite out of range

"""

class SatelliteEnv(Env):
    def __init__(self):
        # A = {transmit, not transmit} = {1, 0}
        self.action_space = Discrete(2)
        # S = (P, S1, S2, S3, E, O, T)
        #Note currently only using P,S1,O
        self.observation_space = Tuple((Box(0, 100, shape=(2,)),MultiDiscrete([2])))  # (P, S1, S2, S3, E, O, T)
        # Set start states
        self.state = np.zeros(3)
        for i in range(3): 
            if i<2:
                self.state[i] = 100
            elif i<6:
                self.state[i] = 1
        
        #Transmission Time 60 mins
        self.transmission_time = 60
        
    def step(self, action):
        #A = {transmit, not transmit} = {1, 0}
        #Update States

        if self.state[0]>=5: #no action 1 if P < 5 
            if action==1:
                self.state[0] += -5   #discharging
                self.state[1] += -3 #update free memory
        if action==0:
            self.state[0] += +3 #charging
            
        #Check States within bounds
        for i in range(2):
            if self.state[i]>100:
                self.state[i] = 100  
            elif self.state[i]< 0:
                self.state[i] = 0
             
        # Reduce transmission time by 1 minute
        self.transmission_time -= 1
        
        # Calculate reward
        reward = 0
        if self.state[0]>=30:
            reward += 2
        else: 
            reward += 0
        if action==1:
            reward += 5
        if self.state[1]==0:
            reward += 100
        
        # Check if transmission time is done
        if self.transmission_time <= 0:
            self.state[2] = 0
    
        if 0 in self.state[[1,2]]:
            done = True
        else:
            done = False
        
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset states
        self.state = np.zeros(3)
        for i in range(3): 
            if i<2:
                self.state[i] = 100 #random.randint(90,100)
            else:
                self.state[i] = 1
        # Reset time
        self.transmission_time = 60 
        return self.state
    

In [114]:
env = SatelliteEnv()

In [111]:
env.observation_space.sample()

(array([21.917253, 16.384315], dtype=float32), array([1], dtype=int64))

### S_lookup function

In [115]:
P = np.arange(0,101,1)
S1 = np.append([0],np.arange(1,101,3))
O = np.arange(2)
A = np.arange(2)
state = np.array([P,S1,O])
N = np.size(state[0])*np.size(state[1])*np.size(state[2])
S_lookup = np.zeros([N,3])
i = 0

for l in range(np.size(state[0])):
    for m in range(np.size(state[1])):
        for n in range(np.size(state[2])):
            S_lookup[i] = [state[0][l],state[1][m],state[2][n]]
            i = i + 1
S_lookup = S_lookup.astype(int)
dims = S_lookup.max (0)+1

C:\Users\ricky\AppData\Local\Temp/ipykernel_43192/1700741764.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  state = np.array([P,S1,O])


### Data Collection (Epsilon-Greedy)

In [116]:
alpha = 0.99
eps = 0.9
B = [[1, 1], [1, 1]] # list of beta distributions
num_samples = 100000
data = np.zeros((num_samples, 4))

def beta_update(a, r):
    B[a][0] += r/10.0
    B[a][1] += 10.0-r/10.0

def epsilon_greedy(epsilon):
    if random.random() < epsilon:
        return random.randint(0, 1)
    else:
        return np.argmax([np.random.beta(B[0][0], B[0][1]), np.random.beta(B[1][0], B[1][1])])
    

for i in range(num_samples):
    a = epsilon_greedy(eps)
    s = env.state
    s = (np.rint(s)).astype(int)
    sp, r, d, _ = env.step(a)
    sp = (np.rint(sp)).astype(int)
    beta_update(a, r)
    if d:
        env.reset()
    data[i,:] = [np.where(np.in1d(np.ravel_multi_index(S_lookup.T,dims),np.ravel_multi_index(s.T,dims)))[0][0], a, r, np.where(np.in1d(np.ravel_multi_index(S_lookup.T,dims),np.ravel_multi_index(sp.T,dims)))[0][0]]
    eps = alpha*eps

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

### Implementing Q Learning

In [124]:
Q = np.zeros((7070, 2))
gamma = 1
alpha = 1e-2

def Q_update(Q, s, a, r, sp):
    Q[int(s),int(a)] += alpha*(r + gamma*np.max(Q[int(sp),:]) - Q[int(s),int(a)])
    return Q

def batch_update(Q, k_max):
    for k in range(k_max):
        for i in range(data.shape[0]):
            Q = Q_update(Q, data[i,0], data[i,1], data[i,2], data[i,3])
    print(k_max)
    return Q

def get_policy(Q):
    pol = np.zeros((7070))
    for i in range(len(pol)):
                pol[i] = np.argmax(Q[i,:])
    return pol

Q = batch_update(Q, 200)
pol = get_policy(Q)
print(pol)

200
[0. 0. 0. ... 1. 0. 0.]


In [126]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    a=[]
    action = 1
    while not done:
        #env.render()
        #action = env.action_space.sample()
        a.append(action)
        n_state, reward, done, info = env.step(action)
        p, s1, o = n_state
        score+=reward
        n_state_3 = np.array([int(p+3), int(s1), int(o)])
        n_state_idx = np.where(np.in1d(np.ravel_multi_index(S_lookup.T,dims),np.ravel_multi_index(n_state_3.T,dims)))[0][0]
        action = pol[n_state_idx]
    out_states = ','.join(map('{:6}'.format, n_state[[0,1,2]]))
    print('Episode:{0:3}   Score:{1:5,.1f}   Transmissions:{2:4}   Total Time:{3:4}   Final States(P,S,O):{4:5}'.format(episode, score, a.count(1), np.size(a), out_states))

Episode:  1   Score:325.0   Transmissions:  59   Total Time:  60   Final States(P,S,O):   3.0,  40.0,   0.0
Episode:  2   Score:325.0   Transmissions:  59   Total Time:  60   Final States(P,S,O):   3.0,  40.0,   0.0
Episode:  3   Score:325.0   Transmissions:  59   Total Time:  60   Final States(P,S,O):   3.0,  40.0,   0.0
Episode:  4   Score:325.0   Transmissions:  59   Total Time:  60   Final States(P,S,O):   3.0,  40.0,   0.0
Episode:  5   Score:325.0   Transmissions:  59   Total Time:  60   Final States(P,S,O):   3.0,  40.0,   0.0
